In [9]:
import os
import json
import requests
from tqdm import tqdm
from time import sleep


# base url to reach sim
base_url = 'http://localhost:8080/'

In [8]:
i=0
tasks = ["stack", "L", "pyramid"]
for t in tasks:
    print("Running task: ", t)
    task_folder = f'data/llm_responses/{t}'
    for file in sorted(os.listdir(task_folder))[7:8]:
        print(f"\nRunning query {i+1}: {file}")
        # reset env
        if i != 0: requests.get(base_url + 'reset')
        
        # load data
        data = json.load(open(f"{task_folder}/{file}", 'r'))
        
        query = data['query']
        plan = data['plan']
        optimizations = data['optimizations']

        # upload plan and optimizations
        requests.post(base_url+'upload_plan', json={"query": query, "plan": plan})
        requests.post(base_url+'upload_optimizations', json={"content": optimizations})

        response = ""
        while response != "finished":
            response = requests.get(base_url+'next_task').json()[-1]['content']
            sleep(3)

        # update counter
        i+=1

_ = requests.get(base_url + 'close')

Running task:  stack

Running query 1: 15.json
Running task:  L

Running query 2: 15.json
Running task:  pyramid

Running query 3: 15.json


In [10]:
task_folder = f'data/llm_responses/stack'
data = json.load(open(f"{task_folder}/3.json", 'r'))